# Fuego contra Fuego

> Fuego contra fuego es amar.

Ricky Martin

Instalamos, cargamos y seteamos el entorno

In [20]:
#%pip install scikit-learn==1.3.2
#%pip install seaborn==0.13.1
#%pip install numpy==1.26.4
#%pip install matplotlib==3.7.1
#%pip install optuna==3.6.1

In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier, plot_tree,  _tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

from joblib import Parallel, delayed

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

In [22]:
dataset_path = '../../../datasets/'
modelos_path = '../../../modelos/'
db_path = '../../../db/'
dataset_file = 'competencia_01.csv'

ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = 202102
mes_test = 202104

# agregue sus semillas
semillas = [211777, 174989, 131497, 612223, 234803]

data = pd.read_csv(dataset_path + dataset_file)

Seguimos trabajando con Febrero como entrenamiento y Abril como test.

In [23]:
X = data[data['foto_mes'] == mes_train]
y = X['clase_ternaria']
X = X.drop(columns=['clase_ternaria'])

In [ ]:
X_futuro = data[data['foto_mes'] == mes_test]
y_futuro = X_futuro['clase_ternaria']
X_futuro = X_futuro.drop(columns=['clase_ternaria'])

Y variaremos la forma de la función de ganancia, para poder ser utilizada de una forma más genérica.

In [25]:
def ganancia_prob(y_hat, y, prop=1, class_index=1, threshold=0.025):
  @np.vectorize
  def ganancia_row(predicted, actual, threshold=0.025):
    return  (predicted >= threshold) * (ganancia_acierto if actual == "BAJA+2" else -costo_estimulo)

  return ganancia_row(y_hat[:,class_index], y).sum() / prop

Ajustamos los modelos de la clase pasada.

In [ ]:
param_ale = {'max_depth': 5,
             'min_samples_split': 80}

param_opt = {'criterion': 'entropy',
             'max_depth': 20,
             'min_samples_split': 145,
             'min_samples_leaf': 14,
             'max_leaf_nodes': 13}

model_base = DecisionTreeClassifier(random_state=semillas[0])
model_ale = DecisionTreeClassifier(random_state=semillas[0], **param_ale)
model_opt = DecisionTreeClassifier(random_state=semillas[0], **param_opt)

model_base.fit(X, y)
model_ale.fit(X, y)
model_opt.fit(X, y)

In [ ]:
y_pred_base = model_base.predict_proba(X_futuro)
y_pred_ale = model_ale.predict_proba(X_futuro)
y_pred_opt = model_opt.predict_proba(X_futuro)

Recordemos la ganancia de cada uno en Abril

In [ ]:
print(f"Ganancia de modelo Base: {ganancia_prob(y_pred_base, y_futuro)}")
print(f"Ganancia de modelo Ale: {ganancia_prob(y_pred_ale, y_futuro)}")
print(f"Ganancia de modelo Opt: {ganancia_prob(y_pred_opt, y_futuro)}")

Antes de continuar con nuestro camino de estrés y sumeración, analizaremos que tan bien hubieramos elegido un modelo de acuerdo a utilizar un leaderboard público. Primero comparamos el **base** y el **ale**

In [ ]:
sss_futuro = StratifiedShuffleSplit(n_splits=50,
                             test_size=0.3,
                             random_state=semillas[0])
modelos = {"base": y_pred_base, "ale": y_pred_ale}
rows = []
for private_index, public_index in sss_futuro.split(X_futuro, y_futuro):
  row = {}
  for name, y_pred in modelos.items():
    row[name + "_private"] = ganancia_prob(y_pred[private_index], y_futuro.iloc[private_index], 0.7)
    row[name + "_public"] = ganancia_prob(y_pred[public_index], y_futuro.iloc[public_index], 0.3)
  rows.append(row)
df_lb = pd.DataFrame(rows)

Una forma de ver si una distribución es distinta a otra es usar el test de wilcoxon. Este test se usa para determinar si hay una diferencia significativa en las medianas de dos muestras dependientes.

Lo vamos a aplicar sobre nuestro simulado leaderboard público con la esperanza que nos ayude a eligir cual de los dos es mejor, para esa muestra.

(Recuerde, no se esta aplicando al Leaderboard público real. Técnicamente estamos aplicandolo a un out of time sample)

In [ ]:
from scipy.stats import wilcoxon

diff_public = df_lb['base_public'] - df_lb['ale_public']
_, p_value = wilcoxon(diff_public)

print(f"p-value: {p_value}")

Espero que sus recuerdos traumaticos de estadísticas le ayuden a leer que la prueba plantea que ambas distribuciones son lo diferentes, y **ale** es mayor.

Veremos con el privado que tan bien nos hubiera ido.

In [ ]:
df = pd.DataFrame()

df['best_public'] = df_lb.filter(regex='_public').idxmax(axis=1)
df['best_private'] = df_lb.filter(regex='_private').idxmax(axis=1)

In [ ]:
pd.crosstab(df['best_public'], df['best_private'])

Sus opiniones. Volvemos al panic de la clase pasada, no? Vamos a agregar el modelo optimizado a la comparación

In [ ]:
sss_futuro = StratifiedShuffleSplit(n_splits=50,
                             test_size=0.3,
                             random_state=semillas[0])
modelos = {"opt":y_pred_opt}
rows = []
for private_index, public_index in sss_futuro.split(X_futuro, y_futuro):
  row = {}
  for name, y_pred in modelos.items():
    row[name + "_private"] = ganancia_prob(y_pred[private_index], y_futuro.iloc[private_index], 0.7)
    row[name + "_public"] = ganancia_prob(y_pred[public_index], y_futuro.iloc[public_index], 0.3)
  rows.append(row)
df_temp = pd.DataFrame(rows)
df_lb = pd.concat([df_lb, df_temp], axis=1)

In [ ]:
df['best_public'] = df_lb.filter(regex='_public').idxmax(axis=1)
df['best_private'] = df_lb.filter(regex='_private').idxmax(axis=1)

pd.crosstab(df['best_public'], df['best_private'])


Sus reflexiones.

---
.

.

.

.

.

.

---

Creo que necesitamos algo superador. Para eso nos abrazaremos al problema, todo el daño que produce el azar, será la luz que resuelva nuestro problema.

Primero hablamos de ensemables, qué son?

* Un ensemble de modelos es una técnica donde se combinan múltiples modelos individuales para mejorar la precisión y robustez de las predicciones. La idea es que al combinar varios modelos, se pueden aprovechar las fortalezas de cada uno y reducir la posibilidad de errores que podría cometer un único modelo.

* **Tipos de ensemble**:
 * **Bagging (Bootstrap Aggregating)**: Consiste en entrenar varios modelos base en diferentes subconjuntos del conjunto de datos de entrenamiento obtenidos mediante técnicas de remuestreo como el bootstrap y luego promediar sus predicciones. Ejemplo: **Random Forest**
 * **Boosting**: En esta técnica, los modelos se entrenan de manera secuencial. Cada modelo intenta corregir los errores cometidos por el modelo anterior. Ejemplo: AdaBoost y **Gradient Boosting**.
 * **Stacking**: En el stacking, se entrenan varios modelos y se combinan usando un "modelo meta". Las predicciones de los modelos base sirven como features para entrenar este modelo meta, que produce la predicción final.

* **Ventajas de usar ensemble de modelos**:
 * **Mejor rendimiento**: Al combinar modelos, generalmente se mejora la precisión en comparación con un solo modelo.
 * **Robustez**: Al integrar diferentes modelos, se mitiga el riesgo de que los errores de un modelo individual afecten gravemente la predicción final.




Pongamos foco en el **Random Forest**

Es un algoritmo de aprendizaje automático que funciona creando un conjunto de árboles de decisión. Para la creación de **árboles distintos** utiliza una técnica llamada bagging para crear múltiples subconjuntos del conjunto de datos de entrenamiento. Cada subconjunto se genera seleccionando al azar muestras del conjunto de datos original con reemplazo. No usa la totalidad de los datos de entrenamiento para cada conjunto. Los datos que quedan fueran son conocidos como **Out of Bag (oob)**

Para cada subconjunto, se construye un árbol de decisión. Sin embargo en cada nodo del árbol, **Random Forest** selecciona de forma aleatoria un grupo de variables y ajusta el árbol con esas variables. Este proceso ayuda a crear árboles que son menos correlacionados entre sí.

Cada árbol en el bosque se entrena de manera independiente usando su respectivo subconjunto de datos. Lueago, para una nueva observación, cada árbol realiza una predicción. El Random Forest luego combina las predicciones de todos los árboles para hacer una predicción final, devolviendo el promedio de las probabilidades de cada árbol individual.



Como desde la clase pasada solo personas más inteligentes, no vamos a empezar a probar **Random Forest** simples. Vamos a parametrizarlo desde el vamos.

Primero vamos a entender algunas limitaciones de la implementación:

El **Random Forest** no soporta nulos! Shame on you sklearn!.

Vamos a tener que imputar los datos. Discutamos entre todos forma de imputar los datos, mientras para salir del paso usamos la peor de todas.

In [ ]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
Xi = imp_mean.fit_transform(X)

Los parámetros que se pueden ajustar en el **rf** son

1. **n_estimators**: Número de árboles en el bosque.
2. **max_depth**: Profundidad máxima de los árboles.
3. **min_samples_split**: Número mínimo de muestras requeridas para dividir un nodo interno.
4. **min_samples_leaf**: Número mínimo de muestras requeridas para estar en un nodo hoja.
5. **max_features**: Número de features a usar en cada árbol. **sqrt** es una elección histórica.
6. **max_leaf_nodes**: Número máximo de nodos hoja en cada árbol.
7. **oob_score**: Indica si se usa la muestra fuera de bolsa (out-of-bag) para estimar la calidad del modelo. Para evitar hacer un **montecarlo-cross-validation** que se toma su tiempo, usaremos esta opción para buscar el mejor modelo. No es la mejor opción. Pero no es tan mala.
8. **n_jobs**: Siempre -1, para que use todos los cores presentes en 
9. **max_samples**: Fracción de los samples.

Finalmente nuestra función de optimización queda la siguiente forma:

In [ ]:
def objective(trial):
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 2000)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 200)
    max_features = trial.suggest_float('max_features', 0.05, 0.7)

    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        max_samples=0.7,
        random_state=semillas[0],
        n_jobs=-1,
        oob_score=True
    )

    model.fit(Xi, y)

    return ganancia_prob(model.oob_decision_function_, y)

storage_name = "sqlite:///" + db_path + "optimization_tree.db"
study_name = "exp_206_random-forest-opt"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

In [ ]:
study.optimize(objective, n_trials=100)

Exploramos como fue la búsqueda de parámetros

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_contour(study)

In [ ]:
plot_contour(study, params=["max_depth", "min_samples_split"])

Ajustamos el mejor modelo

In [ ]:
model_rf = RandomForestClassifier(
        n_estimators=100,
        **study.best_params,
        max_samples=0.7,
        random_state=semillas[0],
        n_jobs=-1,
        oob_score=True
    )

model_rf.fit(Xi, y)


Guardamos el modelo, para no tener que optimizar cada vez que lo queramos usar.

In [ ]:
filename = modelos_path + 'exp_206_random_forest_model_100.sav'
pickle.dump(model_rf, open(filename, 'wb'))

Y lo cargamos para utilizarlo

In [ ]:
model_rf = pickle.load(open(filename, 'rb'))
model_rf

Ahora vamos medir su ganancia sobre el dataset de **abril**

In [ ]:
y_pred_rf = model_rf.predict_proba(Xif)
ganancias_rf = ganancia_prob(y_pred_rf, y_futuro)
print(f"Ganancia de modelo RF: {ganancias_rf}")

What!? Qué paso? que son esos números???

Algo malo que hicimos, es usar poquitos estimadores. Tan solo unos 100. Suficientes como para que la optimización no tarde una eternidad, pero es muy poco. Deberíamos sumar unos cuantos más.

In [ ]:
model_rf_1000 = RandomForestClassifier(
        n_estimators=1000,
        **study.best_params,
        max_samples=0.7,
        random_state=semillas[0],
        n_jobs=-1,
        oob_score=True
    )

model_rf_1000.fit(Xi, y)

filename_rf_1000 = modelos_path + 'exp_206_random_forest_model_1000.sav'
pickle.dump(model_rf, open(filename_rf_1000, 'wb'))

model_rf_1000 = pickle.load(open(filename_rf_1000, 'rb'))
model_rf_1000

Veamos si sumar 10 veces más estimadores tuvo algún efecto

In [ ]:
y_pred_rf = model_rf_1000.predict_proba(Xif)
ganancias_rf = ganancia_prob(y_pred_rf, y_futuro)
print(f"Ganancia de modelo RF 1000: {ganancias_rf}")

Mejoró! Cada peso vale.

Sin embargo, es este modelo tan superior o será ese número lindo.

Calculemos su impacto en los leaderboard

In [ ]:
sss_futuro = StratifiedShuffleSplit(n_splits=50,
                             test_size=0.3,
                             random_state=semillas[0])
modelos = {"rf":y_pred_rf}
rows = []
for private_index, public_index in sss_futuro.split(X_futuro, y_futuro):
  row = {}
  for name, y_pred in modelos.items():
    row[name + "_private"] = ganancia_prob(y_pred[private_index], y_futuro.iloc[private_index], 0.7)
    row[name + "_public"] = ganancia_prob(y_pred[public_index], y_futuro.iloc[public_index], 0.3)
  rows.append(row)
df_temp = pd.DataFrame(rows)
df_lb = pd.concat([df_lb, df_temp], axis=1)

In [ ]:
df = pd.DataFrame()
df['best_public'] = df_lb.filter(regex='_public').idxmax(axis=1)
df['best_private'] = df_lb.filter(regex='_private').idxmax(axis=1)

pd.crosstab(df['best_public'], df['best_private'])

Vaya! es lo que buscabamos, un modelo del que estar seguros. Podrémos ver esto en los histogramas?

In [ ]:
df_lb_long = df_lb.reset_index()
df_lb_long = df_lb_long.melt(id_vars=['index'], var_name='model_type', value_name='ganancia')
df_lb_long[['modelo', 'tipo']] = df_lb_long['model_type'].str.split('_', expand=True)
df_lb_long = df_lb_long[['ganancia', 'tipo', 'modelo']]

In [ ]:
g = sns.FacetGrid(df_lb_long, col="tipo", row="modelo", aspect=2)
g.map(sns.histplot, "ganancia", kde=True)
plt.show()

Por último evaluamos cuales son las variables más importantes del modelo. ¿Habrá influido la imputación?

In [ ]:
importances = model_rf.feature_importances_

features = X.columns
feat_importances = pd.DataFrame({'feature': features, 'importance': importances})
feat_importances = feat_importances.sort_values('importance', ascending=False)

feat_importances.head(25)

Felicitaciones! ahora sabe como superar un **árbol de decisión**.

Pregunta:
* **¿Cómo sabe que un random forest es superior a otro?**
* **¿Cómo supera el random forest presentado?**

## Tarea:

* Envíe a modelo de **rf** a Kaggle
* Mejore la parametrización del **rf**
* Juegue un poco:
 * Borre la variable más importante y mida el **rf**.
 * Agregue variables 100% aleatorias. ¿Salen dentro de las variables más importante?
 * No probamos en **rf** en el mismo dataset de entrenamiento. ¿Se puede decir que sobre ajusta?

#### 1. Comienzo.

In [1]:
#a. Librerías.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier, plot_tree,  _tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

from joblib import Parallel, delayed

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

c:\Users\c678456\AppData\Local\anaconda3\envs\dmeyf\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#b. Constantes.
mes_train = [202101,202102,202103,202104]
mes_test = 202106

dataset_path = '../../../datasets/'
modelos_path = '../../../modelos/'
db_path = '../../../db/'
dataset_file = 'competencia_01.csv'

ganancia_acierto = 273000
costo_estimulo = 7000

semillas = [211777, 174989, 131497, 612223, 234803]

In [3]:
#c. Lectura.
data = pd.read_csv(dataset_path + dataset_file)

In [4]:
#d. Función de ganancia.
def ganancia_prob(y_hat, y, prop=1, class_index=1, threshold=0.025):
  @np.vectorize
  def ganancia_row(predicted, actual, threshold=0.025):
    return  (predicted >= threshold) * (ganancia_acierto if actual == "BAJA+2" else -costo_estimulo)

  return ganancia_row(y_hat[:,class_index], y).sum() / prop

In [5]:
#e. Separo entre Train y Test.
#i. Configuro Train.
X = data[data['foto_mes'].isin(mes_train)]
y = X['clase_ternaria']
X = X.drop(columns=['clase_ternaria'])
#ii. Configuro Test.
X_futuro = data[data['foto_mes'] == (mes_test)]
y_futuro = X_futuro['clase_ternaria']
X_futuro = X_futuro.drop(columns=['clase_ternaria'])

#### 2. Entrenamiento de Random Forest imputando los nulos con .mean()

In [6]:
#a. Imputo con la Media.
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
Xi = imp_mean.fit_transform(X)
Xif = imp_mean.fit_transform(X_futuro)

In [7]:
#b. Defino la función para la búsqueda de los hiperparámetros.
def objective(trial):
    max_depth = trial.suggest_int('max_depth', 2, 45)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 2000)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 200)
    max_features = trial.suggest_float('max_features', 0.05, 0.8)

    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        max_samples=0.7,
        random_state=semillas[0],
        n_jobs=-1,
        oob_score=True 
    )

    model.fit(Xi, y)

    return ganancia_prob(model.oob_decision_function_, y) # Para evaluar el modelo, en vez de dividir entre Train y Test 5 veces y promediarlos, usa los "Datos fuera de la Bolsa" (OBB).

storage_name = "sqlite:///" + db_path + "optimization_tree.db"
study_name = "exp_206_random-forest-opt"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-09-08 13:56:52,308] Using an existing study with name 'exp_206_random-forest-opt' instead of creating a new one.


In [8]:
#c. Realizo el estudio.
study.optimize(objective,n_trials=100)

[I 2024-09-08 14:05:22,822] Trial 87 finished with value: 412503000.0 and parameters: {'max_depth': 20, 'min_samples_split': 112, 'min_samples_leaf': 20, 'max_features': 0.32534536942321757}. Best is trial 44 with value: 419447000.0.
[I 2024-09-08 14:24:32,315] Trial 88 finished with value: 416535000.0 and parameters: {'max_depth': 17, 'min_samples_split': 58, 'min_samples_leaf': 54, 'max_features': 0.3442602569670227}. Best is trial 44 with value: 419447000.0.
[I 2024-09-08 14:34:42,749] Trial 89 finished with value: 415359000.0 and parameters: {'max_depth': 17, 'min_samples_split': 49, 'min_samples_leaf': 41, 'max_features': 0.3421864396466471}. Best is trial 44 with value: 419447000.0.
[I 2024-09-08 14:41:34,067] Trial 90 finished with value: 410921000.0 and parameters: {'max_depth': 13, 'min_samples_split': 246, 'min_samples_leaf': 52, 'max_features': 0.29045443232756607}. Best is trial 44 with value: 419447000.0.
[I 2024-09-08 14:53:16,652] Trial 91 finished with value: 418509000.

KeyboardInterrupt: 

In [9]:
#d. Analizo la performance para distintos valores de los hiperparámetros, y su resultado en las ganancias.
plot_slice(study)

In [10]:
#e. Construyo el modelo con dichos hiperparámetros.
model_rf = RandomForestClassifier(
        n_estimators=100,
        **study.best_params,
        max_samples=0.7,
        random_state=semillas[0],
        n_jobs=-1,
        oob_score=True
    )

In [11]:
#f. Entreno.
model_rf.fit(Xi, y)

RandomForestClassifier(max_depth=23, max_features=0.40060455200913236,
                       max_samples=0.7, min_samples_leaf=47,
                       min_samples_split=52, n_jobs=-1, oob_score=True,
                       random_state=211777)

In [12]:
#g. Guardo el Modelo.
filename = modelos_path + 'exp_206_rf_100_imputacion_media.sav'
pickle.dump(model_rf, open(filename, 'wb'))

In [ ]:
filename = modelos_path + 'exp_206_rf_100_imputacion_media.sav'
model_rf = pickle.load(open(filename, 'rb'))


In [13]:
#g. Lo cargo nuevamente para usarlo.
model_rf = pickle.load(open(filename, 'rb'))
model_rf

RandomForestClassifier(max_depth=23, max_features=0.40060455200913236,
                       max_samples=0.7, min_samples_leaf=47,
                       min_samples_split=52, n_jobs=-1, oob_score=True,
                       random_state=211777)

In [14]:
#h. Predigo Junio.
#i. Realizo la predicción de probabilidades usando el modelo entrenado.
predicciones = model_rf.predict_proba(X_futuro)
#ii. Encuentro el índice de la columna "BAJA+2".
indice_baja2 = model_rf.classes_.tolist().index("BAJA+2")
#iii. Agrego la columna de probabilidad de "BAJA+2" al DataFrame.
X_futuro['prob_baja2'] = predicciones[:, indice_baja2]
#iv. Solo envío estímulo a los registros con probabilidad de "BAJA+2" mayor a 1/40.
X_futuro['Predicted'] = (X_futuro['prob_baja2'] > 1/40).astype(int)
#v. Selecciono las columnas de interés.
resultados = X_futuro[["numero_de_cliente","Predicted"]].reset_index(drop=True) 
#vi. Exporto como archivo .csv.
nombre_archivo = "K104_001.csv"
ruta_archivo= "../../../exp/{}".format(nombre_archivo)
resultados.to_csv(ruta_archivo,index=False)

c:\Users\c678456\AppData\Local\anaconda3\envs\dmeyf\Lib\site-packages\sklearn\base.py:486: UserWarning:

X has feature names, but RandomForestClassifier was fitted without feature names



In [15]:
#i. Envío a Kaggle.
#a. Importo librería.
from kaggle.api.kaggle_api_extended import KaggleApi
#b. Configura el API de Kaggle
api = KaggleApi()
api.authenticate()
#c. Defino los parámetros claves.
mensaje = f'Archivo {nombre_archivo}. Se predice Junio con modelo entrenado con Enero, Febrero, Marzo y Abril. Imputación con la Media. 100 Trials para búsqueda de hiperparámetros'
competencia = 'dm-ey-f-2024-primera'
#c. Subo la Submission.
api.competition_submit(file_name=ruta_archivo,message=mensaje,competition=competencia)

100%|██████████| 2.08M/2.08M [00:01<00:00, 1.32MB/s]


Successfully submitted to DMEyF 2024 Primera

#### 3. Imputar los nulos de mejor manera.

In [ ]:
#a. Analizo las columnas con nulos.
#i. Número total de filas.
total_filas = data.shape[0]

#ii. Porcentaje de valores nulos en cada columna.
porcentaje_nulos = round((data.isna().sum() / total_filas) * 100, 2)

#iii. Cantidad total de valores nulos por columna.
total_nulos = data.isna().sum()

#iv. Combino los resultados en un DataFrame.
nulos_df = pd.DataFrame({
    'Porcentaje Nulos': porcentaje_nulos,
    'Total Nulos': total_nulos
})

#v. Ordeno por el porcentaje de nulos de mayor a menor, y muestro las primeras 48 columnas.
nulos_df_ordenado = nulos_df.sort_values(by='Porcentaje Nulos', ascending=False).head(48)

#vi. Printeo.
print("En total, el dataset tiene {} filas. Estos son los porcentajes y cantidades de nulos por columna".format(total_filas))
nulos_df_ordenado

In [ ]:
#b. Imputo.
#i. Columnas de mora.
nc_mora = ["Master_Finiciomora","Visa_Finiciomora"]
#----> Le asigno -1 a los nulos ya que no corresponden por falta de mora.
data[nc_mora] = data[nc_mora].fillna(-1)

In [ ]:
#ii. Columnas pertenecientes a TC de entidades que no posee el cliente.
nc_no_tc_master = [
    "Master_mfinanciacion_limite", # Puedo imputar con valores negativos ya que va entre 0 e infinito.
    "Master_status", # Puedo imputar con valores negativos ya que va entre 0 e infinito.
    "Master_delinquency", # Puedo imputar con valores negativos ya que va entre 0 e infinito.
    "Master_mpagado", # Puedo imputar con valores negativos ya que va entre 0 e infinito.
    "Master_mlimitecompra", # Puedo imputar con valores negativos ya que va entre 0 e infinito.
    "Master_fechaalta", # Puedo imputar con 0 o valores negativos ya que va entre 0 e infinito.
    "Master_mpagominimo" # Puedo imputar con valores negativos ya que va entre 0 e infinito.
    "Master_msaldodolares", # Pensar como imputar ya va entre - infinito a + infinito. (0?)
    "Master_msaldototal", # Pensar como imputar ya va entre - infinito a + infinito. (0?)
    "Master_Fvencimiento", # Pensar como imputar ya va entre - infinito a + infinito. (0?)
    "Master_msaldopesos",  # Pensar como imputar ya va entre - infinito a + infinito. (0?)
    
    ] 

nc_no_tc_visa = [
    "Visa_mpagominimo", # Puedo imputar con valores negativos ya que va entre 0 e infinito.
    "Visa_fechaalta", # Puedo imputar con 0 o valores negativos ya que va entre 0 e infinito.
    "Visa_mpagado", # Puedo imputar con valores negativos ya que va entre 0 e infinito.
    "Visa_mlimitecompra", # Puedo imputar con valores negativos ya que va entre 0 e infinito.
    "Visa_mfinanciacion_limite", # Puedo imputar con valores negativos ya que va entre 0 e infinito.
    "Visa_status", # Puedo imputar con valores negativos ya que va entre 0 e infinito.
    "Visa_delinquency"	# Puedo imputar con valores negativos ya que va entre 0 e infinito.
    "Visa_msaldopesos", # Pensar como imputar ya va entre - infinito a + infinito. (0?)
    "Visa_msaldototal", # Pensar como imputar ya va entre - infinito a + infinito. (0?)
    "Visa_Fvencimiento", # Pensar como imputar ya va entre - infinito a + infinito. (0?)
    "Visa_msaldodolares", # Pensar como imputar ya va entre - infinito a + infinito. (0?)

    ]


In [4]:
#iii. Columnas pertenecientes a pagos, consumos, adelantos y pagos.
consumos_generales_master = [
    "Master_mconsumototal",     # Todos los consumos, mas allá de la TC.        
    "Master_mconsumosdolares",  # Todos los consumos, mas allá de la TC.
    "Master_mconsumospesos",    # Todos los consumos, mas allá de la TC.
    "Master_cconsumos",         # Consumos con la TC.
    "Master_mpagosdolares",
    "Master_mpagospesos",     
    "Master_cadelantosefectivo", 
    "Master_madelantopesos",  
    "Master_madelantodolares",  
        
]

consumos_generales_visa = [
    "Visa_mpagosdolares",       
    "Visa_mconsumototal",          
    "Visa_cconsumos",              
    "Visa_cadelantosefectivo",   
    "Visa_madelantodolares",      
    "Visa_madelantopesos",          
    "Visa_mconsumosdolares",         
    "Visa_mconsumospesos",           
    "Visa_mpagospesos"             
]

# caso 1 ----> No tengo la tarjeta, entonces no corresponde, los imputo con -1:
#data[(data[consumos_generales_master].isna().all(axis=1))&(data["ctarjeta_master"]==0)]

# caso 2 ----> Tengo la tarjeta, pero sin movimientos, los imputo con 0.
# La gran mayoría, no tuvo transacciones (25 casos si).
# La gran mayoria, no tuvo consumos con la TC (mtarjeta_master_consumo) (25 casos si).

#Ej: data[(data[consumos_generales_master].isna().all(axis=1))&(data["ctarjeta_master"]==1)]["ctarjeta_master_transacciones"].value_counts()

# Entonces... ¿Cuál es la diferencia con los casos donde = 0 en vez de nulo? No la encuentro.
# data[(data["ctarjeta_master"]==1)&(data["Master_cconsumos"]==0)]["mtarjeta_master_consumo"].value_counts() Tiene la TC, no realizó consumos, pero tiene montos de consumo :O.

In [ ]:
#iv. Columnas de Fecha de último cierre.
nc_ultimo_cierre = [
    "Master_fultimo_cierre",
    "Visa_fultimo_cierre"
]
# ------> Se imputa con -1 ya que no corresponde porque son casos o que no tienen la TC (ctarjeta_master=0),
# o se dieron de alta hace menos de 30 días (Master_fechaalta<30), y por ende, no tuvieron nunca una fecha de cierre todavía.
data[nc_ultimo_cierre] = data[nc_ultimo_cierre].fillna(-1)

In [ ]:
#v. Columnas de Descuentos.
descuentos = [
    "mtarjeta_master_descuentos",
    "mtarjeta_visa_descuentos"
    ]
# ----> Para los casos que no corresponde, se pone 0 y no None.
# De hecho, la gran mayoría de los datos faltantes corresponden a clientes con la Tarjeta de Crédito.
# Al investigar, se ve que todos los casos donde hay datos faltantes del monto del descuento 
# tiene datos de cantidades de descuentos (ctarjeta_master_descuentos).
# Hipótesis: Se tomó un descuento pero no se procesó el monto aún? Error en la carga de datos?
# Vamos a calcular cual es el monto medio de cada descuento, y multiplicar por la columna de cantidad de descuentos.

#### 4. División entre Train y Test.

In [ ]:
#a. Train.
X = data[data['foto_mes'].isin(mes_train)]
y = X['clase_ternaria']
X = X.drop(columns=['clase_ternaria'])

In [ ]:
#b. Test.
X_futuro = data[data['foto_mes'] == mes_test]
y_futuro = X_futuro['clase_ternaria']
X_futuro = X_futuro.drop(columns=['clase_ternaria'])

In [ ]:
#c. Verifico que la columna target de Train no tenga vacios.
print("La columna target de train tiene {} vacíos.".format(y.isna().sum()))

#### 5. Búsqueda de hiperparámetros con 100 trials.

In [ ]:
#a. Defino la función de optimización.
def objective(trial):
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 2000)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 200)
    max_features = trial.suggest_float('max_features', 0.05, 0.7)

    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        max_samples=0.7,
        random_state=semillas[0],
        n_jobs=-1,
        oob_score=True
    )

    model.fit(Xi, y)

    return ganancia_prob(model.oob_decision_function_, y)

storage_name = "sqlite:///" + db_path + "optimization_tree.db"
study_name = "exp_206_random-forest-opt"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

In [ ]:
#b. Optimizo.
study.optimize(objective, n_trials=100)

In [ ]:
#c. Visualización de la búsqueda de parámetros.

In [ ]:
#i. Mejor puntaje en la optimización de los parámetros.
optuna.visualization.plot_optimization_history(study)

In [ ]:
#ii. Importancia de los hiperparámetros.
plot_param_importances(study)

In [ ]:
#iii. Analizo la performance para distintos valores de los hiperparámetros, y su resultado en las ganancias.
plot_slice(study)

In [ ]:
#iv. Visualizo la relación entre los diferentes hiperparámetros.
plot_contour(study)

In [ ]:
#v. Veo la relación entre max_depth y max_leaf_nodes.
plot_contour(study, params=["max_depth", "min_samples_split"])

#### 6. Entrenamiento del Random Forest.

In [ ]:
#a. Construimos el modelo con ciertos parámetros.
model_rf = RandomForestClassifier(
        n_estimators=100,
        **study.best_params,
        max_samples=0.7,
        random_state=semillas[0],
        n_jobs=-1,
        oob_score=True
    )

In [ ]:
#b. Entrenamos.
model_rf.fit(Xi, y)

In [ ]:
#c. Guardamos el modelo.
filename = modelos_path + 'exp_206_random_forest_model_100.sav'
pickle.dump(model_rf, open(filename, 'wb'))

In [ ]:
#d. Cargamos el modelo para reutilizarlo.
model_rf = pickle.load(open(filename, 'rb'))
model_rf

In [ ]:
#h. Predigo Junio.
#i. Realizo la predicción de probabilidades usando el modelo entrenado.
predicciones = model_rf.predict_proba(X_futuro)
#ii. Encuentro el índice de la columna "BAJA+2".
indice_baja2 = model_rf.classes_.tolist().index("BAJA+2")
#iii. Agrego la columna de probabilidad de "BAJA+2" al DataFrame.
X_futuro['prob_baja2'] = predicciones[:, indice_baja2]
#iv. Solo envío estímulo a los registros con probabilidad de "BAJA+2" mayor a 1/40.
X_futuro['Predicted'] = (X_futuro['prob_baja2'] > 1/40).astype(int)
#v. Selecciono las columnas de interés.
resultados = X_futuro[["numero_de_cliente","Predicted"]].reset_index(drop=True) 
#vi. Exporto como archivo .csv.
nombre_archivo = "K104_002.csv"
ruta_archivo= "../../../exp/{}".format(nombre_archivo)
resultados.to_csv(ruta_archivo,index=False)

In [ ]:
#c. Envío a Kaggle.
#a. Importo librería.
from kaggle.api.kaggle_api_extended import KaggleApi
#b. Configura el API de Kaggle
api = KaggleApi()
api.authenticate()
#c. Defino los parámetros claves.
mensaje = f'Archivo {nombre_archivo}. Se predice Junio con modelo entrenado con Enero, Febrero, Marzo y Abril. Imputación pensada. 100 Trials para búsqueda de hiperparámetros'
competencia = 'dm-ey-f-2024-primera'
#c. Subo la Submission.
api.competition_submit(file_name=ruta_archivo,message=mensaje,competition=competencia)

#### 7. Búsqueda de Hiperparámetros con 1000 trials.